In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import time
import requests
import pandas as pd
import urllib.parse
from nltk.tokenize import word_tokenize

In [ ]:
for i in range(1, 18):
  file_path = '/content/drive/MyDrive/lok_sabha_data/NER_stats/LS' + str(i) + '_NER_Stats.csv'
  try:
    df =  pd.read_csv(file_path, lineterminator='\n', usecols = ["entity_text",	"entity_type",	"instances"])
    fdf = df[df["entity_type"] == "GPE"]
    statfilepath = '/content/drive/MyDrive/lok_sabha_data/Location_data/ner_stats_' + str(i) + '_loaction_data.csv'
    fdf.to_csv(statfilepath, columns = ["entity_text", "instances"], index = False)
    print("File done:", str(i))
  except FileNotFoundError as e:
    continue

In [ ]:
locations = list()
frequencies = list()
nouns = pd.read_csv("/content/drive/MyDrive/lok_sabha_data/NER_stats/LS16_Noun_Stats.csv")
pnouns = pd.read_csv("/content/drive/MyDrive/lok_sabha_data/NER_stats/LS16_PropN.csv")
ner_loc = pd.read_csv("/content/drive/MyDrive/lok_sabha_data/Location_data/ner_stats_16_loaction_data.csv")

locations += list(nouns["word"]) + list(pnouns["word"]) + list(ner_loc["entity_text"])
frequencies += list(nouns["instances"]) + list(pnouns["instances"]) + list(ner_loc["instances"])

location_df = pd.DataFrame(list(zip(locations, frequencies)), columns =['locations', 'frequencies'])

In [ ]:
print(locations[:100])

In [ ]:
location_df.head()

In [ ]:

coordinates = list()
frequencies = dict()
notFound = list()

start_time = time.time()

for index,loc in enumerate(location_df['location'].values):
        time.sleep(1.2)
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(loc) +'?format=json'
        retry_time = 0.8
        response = None
        while not response:
            try:
                response = requests.get(url).json()
                if response:
                        print(index, "found")
                        coordinates.append([response[0]['display_name'], response[0]['lat'], response[0]['lon']])
                        
                        if response[0]['display_name'] in frequencies.keys():
                                frequencies[response[0]['display_name']] += location_df.iloc[index]['frequency']
                        else:
                                frequencies[response[0]['display_name']] = location_df.iloc[index]['frequency']
                else:
                        notFound.append(index)
                        print(index, "not found")
                print(f"Total : {index+1-len(notFound)}/{index+1}")
                break
            except:
                time.sleep(retry_time)
                print("retrying")
        
        print("Time elapsed :", time.time()-start_time)
        print("\n----------------------------------\n")

In [ ]:
len(frequencies)

In [ ]:
locations

In [ ]:
frequencies

In [ ]:
plot_data = pd.DataFrame(coordinates, columns=['Location', 'Latitude', 'Longitude'])
plot_data.to_csv('locations\LS16_plot_data.csv')

In [ ]:
plot_data = pd.read_csv(f'locations\LS16_plot_data.csv')
print(len(plot_data))
plot_data.head(20)

In [ ]:
plot_data.drop_duplicates(subset=['Location'], inplace=True, ignore_index=True)

In [ ]:
len(plot_data)

In [ ]:
plot_data.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
plot_data

In [ ]:
plot_data.to_csv('locations\LS16_plot_data.csv')

In [ ]:
plot_data = pd.read_csv(f'locations\LS16_plot_data.csv')
print(len(plot_data))
plot_data.head(20)

In [ ]:
plot_data.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
freqs = list()

for loc in plot_data['Location'].values:
    freqs.append(frequencies[loc])

In [ ]:
print(len(freqs))
freqs

In [ ]:
plot_data.insert(3, "Frequency", freqs, True)

In [ ]:
plot_data

In [ ]:
plot_data.to_csv('locations\LS16_plot_data.csv')

In [ ]:
notFound

In [ ]:
not_found_locations = list()

for index in notFound:
    not_found_locations.append(locations.iloc[index]['location'])

In [ ]:
not_found_locations

In [ ]:
not_found = pd.DataFrame()
not_found.insert(0, "index", notFound, True)
not_found.insert(1, "location", not_found_locations, True)

In [ ]:
not_found

In [ ]:
not_found.to_csv('locations\LS16_not_found.csv')

## **Trying to make the location data proper**

In [ ]:
plotData = pd.read_csv('locations\LS16_plot_data.csv')

In [ ]:
plotData

In [ ]:
indexes = list()

for index in range(0, len(plotData)):
    actual = word_tokenize(plotData.iloc[index]['actual_address'].lower())
    formatted = word_tokenize(plotData.iloc[index]['formatted_address'].lower())
    flag = True

    for token in actual:
        if token in formatted:
            flag = False
            break
    
    if flag:
        indexes.append(index)

In [ ]:
print(len(indexes))

In [ ]:
for index in indexes:
    print(list(plotData.iloc[index]))

In [ ]:
import pandas as pd
plot_data = pd.read_csv(f'locations\LS16_plot_data.csv')

In [ ]:
plot_data

In [ ]:
plot_data[4790:4800]

In [ ]:
plot_data.drop(4794)[4790:4800]

In [ ]:
plot_data

## **Hindi Locations**

In [ ]:
import json
import time
import pandas as pd
from geosky import geo_plug
from googletrans import Translator

In [ ]:
locations_in_english = list()

In [ ]:
countries = geo_plug.all_CountryNames()
states = json.loads(geo_plug.all_Country_StateNames())
cities = json.loads(geo_plug.all_State_CityNames())

In [ ]:
locations_in_english += countries

In [ ]:
for index in range(0, len(states)):
  locations_in_english += (list(states[index].values())[0])

In [ ]:
for index in range(0, len(cities)):
  locations_in_english += (list(cities[index].values())[0])

In [ ]:
locations_in_english[:20]

In [ ]:
locations_in_hindi = list()

In [ ]:
locations_in_hindi

In [ ]:
translator = Translator()
start_time = time.time()
from_lang = 'en'
to_lang = 'hi'

data = pd.DataFrame(columns=['English', 'Hindi'])

for index,loc in enumerate(locations_in_english):
    tries = 0
    while(tries<7):
      try:
        result = translator.translate(loc, src=from_lang, dest=to_lang).text

        if result:
          locations_in_hindi.append(result)
          data.loc[len(data)] = [loc, locations_in_hindi[-1]]
          print(f'{index} Done!')
        else:
          print(f'Error in {index}')

        if index%1000==0:
          data.to_csv('translatedLocations.csv')
        
        tries=100
      
      except:
        tries += 1
        time.sleep(15)
        print("Retrying")

print(time.time()-start_time)

In [ ]:
len(data)

In [ ]:
data.to_csv('translatedLocations.csv')

In [ ]:
my_file = open("hindiLocations.txt", "r", encoding="utf8")
content = my_file.read()
my_file.close()

In [ ]:
locations_in_hindi = content.split('\n')

In [ ]:
locations_in_hindi = locations_in_hindi[:-1]

In [ ]:
len(locations_in_hindi)